In [4]:
import os

import torch
from torch import nn
import pandas as pd
from config import config
from dataset import FackFaceDetLoader
from model import FackFaceDetModel

from glob import glob

df = pd.read_csv(os.path.join(config.root, 'submission.csv'), sep='\t')
df['image_path'] = df['fnames'].apply(lambda x: os.path.join(config.root, 'image', 'test', x))
df.head()

,fnames,label,image_path
0,test_0.jpg,1,../datasets/fackface_det/image/test/test_0.jpg
1,test_1.jpg,0,../datasets/fackface_det/image/test/test_1.jpg
2,test_2.jpg,1,../datasets/fackface_det/image/test/test_2.jpg
3,test_3.jpg,0,../datasets/fackface_det/image/test/test_3.jpg
4,test_4.jpg,1,../datasets/fackface_det/image/test/test_4.jpg


In [ ]:
import gc
from tqdm import tqdm
import numpy as np

test_loader = FackFaceDetLoader(df, df, config).val_dataloader()

models_dir = ''
device = "cuda" 

predicted_labels = None
for model_name in glob.glob(models_dir + '/*.pth'):

    model = FackFaceDetModel(config)
    model = nn.DataParallel(model)
    model.load_state_dict(torch.load(model_name,map_location="cuda:0"))
    model = model.cuda()
    model.eval()
    
    temp_preds = None
    with torch.no_grad():
        for (images, target) in tqdm(test_loader, desc=f'Predicting. '):
            images = images.to("cuda", non_blocking=True)
            dense = dense.to("cuda", non_blocking=True)
            predictions = torch.sigmoid(model(images)).to('cpu').numpy()
            
            if temp_preds is None:
                temp_preds = predictions
            else:
                temp_preds = np.vstack((temp_preds, predictions))

    if predicted_labels is None:
        predicted_labels = temp_preds
    else:
        predicted_labels += temp_preds
    
#     del model
    gc.collect()


In [ ]:
sub_df = pd.DataFrame()
sub_df['fnames'] = df['fnames']
sub_df['label'] = predicted_labels